<a href="https://colab.research.google.com/github/harshdsdh/MLstuff/blob/master/tensorflowDebiasingembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#create dataset of all labelled words
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import IPython
import functools
import matplotlib.pyplot as plt
from tqdm import tqdm

In [0]:
from gensim.models import KeyedVectors

In [8]:
emb = KeyedVectors.load_word2vec_format('glove.txt')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def make_gender_words_set(f,emb):
  df = pd.read_csv(f)
  f_words = df.iloc[:,0].values.tolist()
  m_words = df.iloc[:,1].values.tolist()
  return f_words,m_words

In [0]:
def make_stereotype_words_set(f,emb):
  df = pd.read_csv(f,sep='\t')
  f_words = df.iloc[:,0].values.tolist()
  m_words = df.iloc[:,1].values.tolist()
  return f_words,m_words

In [0]:
def make_no_gender_words_set(f,emb):
  df = pd.read_csv(f,sep='\t')
  w = df.iloc[:,0].values.tolist()
  return w

In [0]:
f,m = make_gender_words_set('gender_pair.csv',emb)
arr=[]
arr+=f
arr+=m
arr=[i for i in arr if i in emb]
label = [1 for i in range(len(arr))]
data = [[i,v] for i,v in zip(arr,label)]
main_df = pd.DataFrame(data,columns=['words','labels'])

In [71]:
print(main_df)

           words  labels
0        witches       1
1    maidservant       1
2        mothers       1
3           diva       1
4        actress       1
..           ...     ...
388    paternity       1
389     roosters       1
390      priests       1
391       busboy       1
392        heros       1

[393 rows x 2 columns]


In [0]:
f,m = make_stereotype_words_set('stereotype_list.tsv',emb)
arr=[]
arr+=f
arr+=m
arr=[i for i in arr if i in emb]
label = [0 for i in range(len(arr))]
data = [[i,v] for i,v in zip(arr,label)]
main_df1 = pd.DataFrame(data,columns=['words','labels'])

In [76]:
print(main_df1)

         words  labels
0      apparel       0
1    assistant       0
2        baker       0
3      bathing       0
4    beautiful       0
..         ...     ...
161    science       0
162  scientist       0
163   sculptor       0
164       blue       0
165   football       0

[166 rows x 2 columns]


In [0]:
w = make_no_gender_words_set('no_gender_list.tsv',emb)
arr=[]
arr+=w
arr=[i for i in arr if i in emb]
label = [0 for i in range(len(arr))]
data = [[i,v] for i,v in zip(arr,label)]
main_df2 = pd.DataFrame(data,columns=['words','labels'])

In [81]:
print(main_df2)

          words  labels
0         abate       0
1      aberrant       0
2       abiding       0
3          able       0
4     abolition       0
...         ...     ...
1025    younger       0
1026   youngest       0
1027       zero       0
1028     zigzag       0
1029       zone       0

[1030 rows x 2 columns]


In [82]:
frames=[main_df,main_df1,main_df2]
df = pd.concat(frames)
print(df)

            words  labels
0         witches       1
1     maidservant       1
2         mothers       1
3            diva       1
4         actress       1
...           ...     ...
1025      younger       0
1026     youngest       0
1027         zero       0
1028       zigzag       0
1029         zone       0

[1589 rows x 2 columns]


In [0]:
df.to_csv('fresh_data.csv',header=True,index=False)

In [95]:
df = df.sample(frac=1)
print(df)

           words  labels
257     chairmen       1
1018       wound       0
29           nun       1
165     brighter       0
298       danger       0
...          ...     ...
164         blue       0
627      meeting       0
266   comparison       0
140        blast       0
953          try       0

[1589 rows x 2 columns]


In [0]:
all_words = df['words']
word2emb={}
for word in all_words:
    word2emb[word] = emb[word]

In [203]:
word2emb['blast']

array([ 1.155490e-01,  3.137470e-01,  5.241530e-01, -3.072210e-01,
       -2.074770e-01,  4.851020e-01, -7.411300e-02, -5.476100e-02,
        2.460000e-02, -4.864780e-01,  2.019200e-01, -1.488760e-01,
        2.969410e-01, -9.146400e-02, -1.593760e-01, -2.245730e-01,
        3.117800e-02,  4.950150e-01,  5.012000e-02,  2.196230e-01,
        4.648000e-02,  7.991000e-03, -3.802820e-01,  1.976080e-01,
        1.234250e-01,  3.156900e-01, -1.938230e-01,  3.464740e-01,
       -3.294070e-01, -3.525920e-01,  6.070920e-01, -1.784900e-02,
        3.816130e-01,  2.621620e-01,  6.975900e-02,  5.848450e-01,
        1.653330e-01, -1.349070e-01,  3.458040e-01,  2.179860e-01,
       -1.126390e-01, -4.273750e-01,  8.023800e-02, -4.558480e-01,
        3.733400e-02,  7.910000e-03,  5.140000e-02,  5.438860e-01,
        2.415340e-01,  6.226760e-01, -3.707200e-02, -5.490730e-01,
        2.783450e-01,  1.764550e-01, -3.149470e-01, -3.753100e-02,
        4.445100e-01, -2.183390e-01,  5.527000e-02, -1.573710e

In [120]:
vocab_size = len(all_words)
print(vocab_size)

1589


In [98]:
for i,v in word2emb.items():
  print(len(v))
  break

300


In [0]:
[word2emb[df.iloc[idx]['words']] for idx in range(4)]

In [0]:
def get_batch(df,word2emb,batch_size):
  inds_x = [i for i in range(vocab_size)]
  selected_inds = np.random.choice(inds_x, size = batch_size)
  sorted_inds = sorted(selected_inds)
  train_x = [word2emb[df.iloc[idx]['words']] for idx in sorted_inds]
  train_y = [df.iloc[idx]['labels'] for idx in sorted_inds]
  x_b = np.reshape(train_x,[batch_size,300])
  y_b= np.reshape(train_y,[batch_size,1])
  return (x_b,y_b)


In [0]:
x_batch, y_batch = get_batch(df, word2emb, batch_size=4)

In [212]:
y_batch

array([[0],
       [0],
       [0],
       [0]])

In [0]:
emb_size=300

def make_standard_encoder(n_outputs=1):
  vocab_size = 1589+1
  emb_size=300
  batch_size=4
  model = tf.keras.Sequential([
           tf.keras.layers.Dense(emb_size,input_shape=(4,300),activation='relu',dtype='float32'),
           tf.keras.layers.Dense(emb_size,activation='relu'),
           tf.keras.layers.Dense(n_outputs),
  ])
  return model
sc= make_standard_encoder()

In [215]:
sc.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 4, 300)            90300     
_________________________________________________________________
dense_38 (Dense)             (None, 4, 300)            90300     
_________________________________________________________________
dense_39 (Dense)             (None, 4, 1)              301       
Total params: 180,901
Trainable params: 180,901
Non-trainable params: 0
_________________________________________________________________


In [197]:
x_batch[0]

array([ 0.361572, -0.32764 , -0.121619, -0.333029, -0.835748, -0.241483,
       -0.267943,  0.102412,  0.475686,  0.668341, -0.314658, -0.596146,
       -0.305112, -0.358897, -0.07791 , -0.002601,  0.075599,  0.32295 ,
        0.355335,  0.614285, -0.658294, -0.044071,  0.083704,  0.144311,
        0.022276,  0.370851, -0.004913,  0.189738,  0.336325, -0.299981,
        0.193976, -0.473404,  0.532159, -0.028713, -0.356244,  0.332024,
        0.129192, -0.098207, -0.633016, -0.58662 ,  0.590908,  0.114909,
       -0.078506,  0.184477, -0.532621,  0.406066, -0.153006,  0.152474,
        0.335604, -0.328165,  0.110117,  0.377287, -0.066352,  0.104374,
       -0.218475, -0.012415, -0.384971, -0.220526, -0.374932, -0.225709,
       -0.072646, -0.171626, -0.277433,  0.019069, -0.316737, -0.070353,
       -0.629686, -0.273949,  0.152595,  0.615744, -0.737369, -0.372878,
        0.24441 ,  0.413418, -0.21138 , -0.173993,  0.25742 , -0.032344,
       -0.189323,  0.656813,  0.233036,  0.045089, 

In [218]:
pred = sc(x_batch)

In [219]:
pred

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.19575174],
       [0.07709941],
       [0.25706697],
       [0.7948129 ]], dtype=float32)>

In [226]:
a = tf.constant(y_batch,dtype='float32')     
a

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [222]:
type(y_batch[0])

numpy.ndarray

In [133]:
pip install livelossplot

In [0]:
pred = 

In [0]:
from livelossplot import PlotLosses
liveloss = PlotLosses()

In [0]:
num_epochs = 4  # keep small to run faster
learning_rate = 5e-4
batch_size =4
optimizer = tf.keras.optimizers.Adam(learning_rate)
l=[]
@tf.function
def standard_train_step(x,y):
  with tf.GradientTape() as tape:
    logits = sc(x)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=logits)
  grads = tape.gradient(loss, sc.trainable_variables)
  optimizer.apply_gradients(zip(grads,sc.trainable_variables))
  return loss
for epoch in range(num_epochs):
  logs={}
  for i in tqdm(range(vocab_size//batch_size)):
    x,y = get_batch(df,word2emb,batch_size)
    y = tf.constant(y,dtype='float32')   
    loss = standard_train_step(x,y)
    print(loss)




In [230]:
(b_x,b_y) = get_batch(df,word2emb,500)
b_y = tf.constant(b_y,dtype='float32') 
y_pred_standard = tf.round(tf.nn.sigmoid(sc.predict(b_x)))
acc_standard = tf.reduce_mean(tf.cast(tf.equal(b_y, y_pred_standard), tf.float32))
print("Standard CNN accuracy on (potentially biased) training set: {:.4f}".format(acc_standard.numpy()))

Standard CNN accuracy on (potentially biased) training set: 0.9860
